In [1]:
#!pip install open3d
#!conda install -c open3d-admin open3d

In [2]:
#Code By Zachary Waynor



In [1]:
import open3d as o3d
import numpy as np
import copy

In [2]:
#pointCloudMatrix = [[0.0, 0, 0],[0, 0, 1],[0, 1, 0],[0, 1, 1],[1, 0, 0],[1, 0, 1],[1, 1, 0],[1, 1, 1]];

#pointCloudMatrix = np.array([[0,0,0]])
#for x in range(51):
   # for y in range(51):
       # for z in range(51):
          #  if (abs(x/50) == 1 or abs(y/50) == 1 or abs(z/50)==1 or abs(x) == 0 or abs(y) == 0 or abs(z)==0):
              #  pointCloudMatrix = np.concatenate((pointCloudMatrix,[[x/3,y/3,z/3]]),axis=0)
#pointCloudMatrix = np.array(np.loadtxt('np_array.txt'))                
pointCloudMatrix = np.array(np.loadtxt('vertices-hdroomfixed-64mation.txt'))            


In [3]:
pointCloudMatrix.shape

(18991, 6)

In [4]:
color = pointCloudMatrix[:,3:6]
pointCloudMatrix= pointCloudMatrix[:,0:3]

pointCloudMatrix.shape



(18991, 3)

In [5]:
def simplistic_mesh_creator_voxel(pointCloudMatrix,suppressOutputs=True):
     #takes a numpy matrix of size [x,3] of points 
     #and uses open3d inorder to create a basic voxel representation of the points
    
    o3dPointCloud = o3d.geometry.PointCloud() #Create pointcloud object
    o3dPointCloud.points = o3d.utility.Vector3dVector(pointCloudMatrix) #Populate with points from numpy matrix
    o3dPointCloud.scale(1 / np.max(o3dPointCloud.get_max_bound() - o3dPointCloud.get_min_bound()),
          center=o3dPointCloud.get_center()) #fix the scale
    #o3dPointCloud.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
    #voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,
                                                            #voxel_size=0.05)
    
    v_size = round(max(o3dPointCloud.get_max_bound()-o3dPointCloud.get_min_bound())*0.005,4) #find Side of each voxel
    
    voxel_grid=o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,voxel_size=v_size)
    # o3d.visualization.draw_geometries([voxel_grid]) #visualization
    
    voxels=voxel_grid.get_voxels()
    vox_mesh = o3d.geometry.TriangleMesh()
    #Generate Meshes from the voxels
    for v in voxels:
        cube = o3d.geometry.TriangleMesh.create_box(width=1, height=1,depth=1)
        cube.paint_uniform_color(v.color)
        cube.translate(v.grid_index, relative=False)
        vox_mesh+=cube
    
    vox_mesh.translate([0.5,0.5,0.5], relative=True)
    vox_mesh.scale(v_size, [0,0,0])
    vox_mesh.translate(voxel_grid.origin, relative=True)
    vox_mesh.merge_close_vertices(0.0000001) 
    if(not suppressOutputs):
        o3d.visualization.draw_geometries([vox_mesh]) #visualization
    o3d.io.write_triangle_mesh("Voxel.ply", vox_mesh)
    return vox_mesh;

    

In [6]:
Out_Vox = simplistic_mesh_creator_voxel(pointCloudMatrix,False)

In [7]:
import matplotlib.pyplot as plt

In [8]:
def simplistic_mesh_creator_poisson(pointCloudMatrix,suppressOutputs=True):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pointCloudMatrix)
    pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()), center=pcd.get_center())
    #o3d.visualization.draw_geometries([pcd])

    pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3))) #remove exiting Normals
    #pcd.estimate_normals() #Estimate new normals base on pointcloud
    radius = 0.1   # 
    max_nn = 30   # 
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn))
    pcd.orient_normals_consistent_tangent_plane(100) #Takes a while but works!
    
    if(not suppressOutputs):
        o3d.visualization.draw_geometries([pcd], point_show_normal=True)
    with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
            mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
                pcd, depth=9)
    if(not suppressOutputs):        
        o3d.visualization.draw_geometries([mesh])
        print('visualize densities')
    densities = np.asarray(densities)
    density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
    density_colors = density_colors[:, :3]
    density_mesh = o3d.geometry.TriangleMesh()
    
    density_mesh.vertices = mesh.vertices
    density_mesh.triangles = mesh.triangles
    density_mesh.triangle_normals = mesh.triangle_normals
    density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
    if(not suppressOutputs):
        o3d.visualization.draw_geometries([density_mesh],mesh_show_back_face=True)
    
    rough_Density_Mesh = copy.deepcopy(density_mesh);
    
    o3d.io.write_triangle_mesh("density_mesh_possian_full.ply", density_mesh)
    
    
    print('remove low density vertices')

    #o3d.visualization.draw_geometries([density_mesh],mesh_show_back_face=True)
    vertices_to_remove = densities < np.quantile(densities, 0.05) #best Density For most cases
    density_mesh.remove_vertices_by_mask(vertices_to_remove)
    print(mesh)
    if(not suppressOutputs):
        o3d.visualization.draw_geometries([density_mesh],mesh_show_back_face=True)
    o3d.io.write_triangle_mesh("density_mesh_possian_reduced.ply", density_mesh)
    return pcd, mesh, rough_Density_Mesh, density_mesh;
   

In [9]:
pointCloud_EstimatedNormals,Out_PosRough,Out_PosRColored,Out_Pos=simplistic_mesh_creator_poisson(pointCloudMatrix,False)

[Open3D DEBUG] Input Points / Samples: 18991 / 18899
[Open3D DEBUG] #   Got kernel density: 0.161 (s), 314.613 (MB) / 314.613 (MB) / 315 (MB)
[Open3D DEBUG] #     Got normal field: 0.0770001 (s), 321.637 (MB) / 321.637 (MB) / 321 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 2.082098e-04 / 3.954112e+00
[Open3D DEBUG] #       Finalized tree: 0.197 (s), 198.832 (MB) / 321.637 (MB) / 324 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.291 (s), 193.355 (MB) / 321.637 (MB) / 324 (MB)
[Open3D DEBUG] #Set point constraints: 0.0569999 (s), 190.637 (MB) / 321.637 (MB) / 324 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 546603 / 239416 / 385273
[Open3D DEBUG] Memory Usage: 190.637 MB
[Open3D DEBUG] # Linear system solved: 0.643 (s), 205.984 (MB) / 321.637 (MB) / 324 (MB)
[Open3D DEBUG] Got average: 0.0190001 (s), 190.883 (MB) / 321.637 (MB) / 324 (MB)
[Open3D DEBUG] Iso-Value: 5.106811e-01 = 9.698345e+03 / 1.899100e+04
[Open3D DEBUG] #          Total Solve:       2.7 (s),     3

In [10]:
def generateBallPivot(pointCloud_With_Estimated_Normals,suppressOutputs=True):
    radii = [0.005, 0.01, 0.02, 0.04,0.08]
    rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        pointCloud_With_Estimated_Normals, o3d.utility.DoubleVector(radii))
    if(not suppressOutputs):
        o3d.visualization.draw_geometries([pointCloud_With_Estimated_Normals, rec_mesh])
    o3d.io.write_triangle_mesh("BallPivot.ply", rec_mesh)
    return rec_mesh

In [11]:
Out_BallPivot = generateBallPivot(pointCloud_EstimatedNormals,False);

In [12]:
def generateConvexHull(pointCloudMatrix,suppressOutputs=True):

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pointCloudMatrix)
    pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()), center=pcd.get_center())

    hull, _ = pcd.compute_convex_hull()
    hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
    hull_ls.paint_uniform_color((0, 0, 1))
    if(not suppressOutputs):
        o3d.visualization.draw_geometries([hull,hull_ls],mesh_show_back_face=True)
    o3d.io.write_triangle_mesh("ConvexHull.ply", hull)
    return hull;

In [13]:
Out_Hull = generateConvexHull(pointCloudMatrix,False)

In [14]:
import os
import sys
import torch
!pip install pytorch3d
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes,cubify
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
    point_mesh_face_distance
)
import numpy as np
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from tqdm.notebook import tqdm

In [15]:
# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

In [16]:

src_obj = os.path.join('voxel.obj')
verts, faces, aux = load_obj(src_obj)
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)
src_mesh = Meshes(verts=[verts], faces=[faces_idx])


trg_mesh = torch.tensor([pointCloudMatrix.tolist()])
target_point_count = trg_mesh.size(dim=1)

sample_trg = trg_mesh
sample_src = sample_points_from_meshes(src_mesh, 5000)


loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)

print(loss_chamfer);

tensor(2626.2751)


In [17]:

Out_Hull_Translated = Out_Hull
Out_Hull_Translated.translate([1.5,0,0])
Out_BallPivot.translate([1.5*2,0,0])
Out_PosRough.translate([1.5*3,0,0])
Out_PosRColored.translate([1.5*4,0,0])
Out_Pos.translate([1.5*5,0,0])


o3d.visualization.draw_geometries([Out_Vox, Out_Hull_Translated,Out_BallPivot,Out_PosRough,Out_PosRColored,Out_Pos],mesh_show_back_face=True)
Out_Hull_Translated.translate([-1.5,0,0])
Out_BallPivot.translate([1.5*-2,0,0])
Out_PosRough.translate([-1.5*3,0,0])
Out_PosRColored.translate([-1.5*4,0,0])
Out_Pos.translate([-1.5*5,0,0])



TriangleMesh with 50469 points and 99719 triangles.

In [18]:
#indiviudaul Display For Demo: Vox
o3d.visualization.draw_geometries([Out_Vox],mesh_show_back_face=True)

In [19]:
#indiviudaul Display For Demo: Hull
o3d.visualization.draw_geometries([Out_Hull],mesh_show_back_face=True)

In [20]:
#indiviudaul Display For Demo: Estimated Normals:
o3d.visualization.draw_geometries([pointCloud_EstimatedNormals], point_show_normal=True)

In [21]:
#indiviudaul Display For Demo: BallPivot
o3d.visualization.draw_geometries([Out_BallPivot],mesh_show_back_face=True)

In [22]:
#indiviudaul Display For Demo: Poisson
o3d.visualization.draw_geometries([Out_PosRColored],mesh_show_back_face=True)

In [23]:
#indiviudaul Display For Demo: Poisson Removed Low Densities
o3d.visualization.draw_geometries([Out_Pos],mesh_show_back_face=True)

In [24]:
def MultipleObjectReconstruction(dataset_name,suppressOutputs=True,suppressOutputs2=True):
    FullListOfOutputs = []
    FullListChamferLoss = []

    i=0
    for dataset_selector in dataset_name:
        data = np.loadtxt(f"../utils/dataset/{dataset_selector}.txt")

        #DECIMATION_CONSTANT = 64
        DECIMATION_CONSTANT = 1

        decimated_vert = []

        for i in range(len(data)):
            if(i % DECIMATION_CONSTANT == 0):
                decimated_vert.append(data[i])
        print(len(decimated_vert))
        pointCloudMatrix = np.array(np.asarray(decimated_vert)  )    
        pointCloudMatrix.shape
        color = pointCloudMatrix[:,3:6]
        pointCloudMatrix= pointCloudMatrix[:,0:3]
        pointCloudMatrix.shape
        Out_Vox = simplistic_mesh_creator_voxel(pointCloudMatrix)
        pointCloud_EstimatedNormals1,Out_PosRough,Out_PosRColored,Out_Pos=simplistic_mesh_creator_poisson(pointCloudMatrix)
        Out_BallPivot = generateBallPivot(pointCloud_EstimatedNormals1);
        Out_Hull = generateConvexHull(pointCloudMatrix)
        o3dPointCloud = o3d.geometry.PointCloud() #Create pointcloud object
        o3dPointCloud.points = o3d.utility.Vector3dVector(pointCloudMatrix) #Populate with points from numpy matrix
        o3dPointCloud.scale(1 / np.max(o3dPointCloud.get_max_bound() - o3dPointCloud.get_min_bound()),
              center=o3dPointCloud.get_center())
        
        if(not suppressOutputs):
            o3dPointCloud.translate([-1.5,0,0])
            Out_Hull.translate([1.5,0,0])
            Out_BallPivot.translate([1.5*2,0,0])
            Out_PosRough.translate([1.5*3,0,0])
            Out_PosRColored.translate([1.5*4,0,0])
            Out_Pos.translate([1.5*5,0,0])
            o3d.visualization.draw_geometries([Out_Vox, 
                                               Out_Hull,
                                               Out_BallPivot,
                                               Out_PosRough,
                                               Out_PosRColored,
                                               Out_Pos,
                                               o3dPointCloud
                                              ], mesh_show_back_face=True)
            o3dPointCloud.translate([1.5,0,0])
            Out_Hull.translate([-1.5,0,0])
            Out_BallPivot.translate([1.5*-2,0,0])
            Out_PosRough.translate([-1.5*3,0,0])
            Out_PosRColored.translate([-1.5*4,0,0])
            Out_Pos.translate([-1.5*5,0,0])
            
            

        
       
        FullListOfOutputs.insert(0+(i*6), o3dPointCloud)
        FullListOfOutputs.insert(1+(i*6), Out_Hull)
        FullListOfOutputs.insert(2+(i*6), Out_BallPivot)
        FullListOfOutputs.insert(3+(i*6), Out_PosRough)
        FullListOfOutputs.insert(4+(i*6), Out_PosRColored)
        FullListOfOutputs.insert(5+(i*6), Out_Pos)
        i = i+1
        
    if(not suppressOutputs2):
        
        for x in range(len(dataset_name)):
            for y in range(6):
                FullListOfOutputs[y+(x*6)].translate([1.5*y,0,-1.5*x])



            o3d.visualization.draw_geometries(FullListOfOutputs, mesh_show_back_face=True)   


        for x in range(len(dataset_name)):
               for y in range(6):
                    FullListOfOutputs[y+(x*6)].translate([-1.5*y,0,+1.5*x])

    
    return FullListOfOutputs


In [25]:
#dataset_name = ["office_1","office_2","copyRoom_1","pantry_1","conferenceRoom_1"]
dataset_name = ["vertices-schair-office","vertices-sroom-bed","vertices-sroom-full","vertices-schair-folding"]
#dataset_name = ["vertices-schair-office","vertices-sroom-bed","vertices-sroom-full","vertices-schair-folding","decimated_hq_room_half",'vertices-hdroomfixed-64mation']
FullListOfOutputs = MultipleObjectReconstruction(dataset_name,True,True)


29552
[Open3D DEBUG] Input Points / Samples: 29552 / 25971
[Open3D DEBUG] #   Got kernel density: 0.139 (s), 322.422 (MB) / 322.422 (MB) / 575 (MB)
[Open3D DEBUG] #     Got normal field: 0.164 (s), 339.867 (MB) / 339.867 (MB) / 575 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 5.513932e-05 / 1.629477e+00
[Open3D DEBUG] #       Finalized tree: 0.265 (s), 361.629 (MB) / 361.629 (MB) / 575 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.32 (s), 352.434 (MB) / 361.629 (MB) / 575 (MB)
[Open3D DEBUG] #Set point constraints: 0.0640001 (s), 343.727 (MB) / 361.629 (MB) / 575 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 728379 / 627120 / 205313
[Open3D DEBUG] Memory Usage: 343.727 MB
[Open3D DEBUG] # Linear system solved: 0.627 (s), 368.246 (MB) / 368.246 (MB) / 575 (MB)
[Open3D DEBUG] Got average: 0.017 (s), 343.195 (MB) / 368.246 (MB) / 575 (MB)
[Open3D DEBUG] Iso-Value: 4.980209e-01 = 1.471751e+04 / 2.955200e+04
[Open3D DEBUG] #          Total Solve:       2.9 (s),     401.

In [26]:

def GetVisualization(ObjectIndex,TypeIndex):
     # 0:o3dPointCloud
     # 1:Out_Hull
     # 2:Out_BallPivot
     # 3:Out_PosRough
     # 4:Out_PosRColored
     # 5:Out_Pos
    o3d.visualization.draw_geometries([FullListOfOutputs[TypeIndex +(6*ObjectIndex)]],mesh_show_back_face=True)
    
def GetGroupVisualization(ObjectIndex):
    x=ObjectIndex
    sublist = []
    for y in range(6):
        FullListOfOutputs[y+(x*6)].translate([1.5*y,0,-1.5*x])
        sublist.insert(y,FullListOfOutputs[y+(x*6)])
        
    o3d.visualization.draw_geometries(sublist, mesh_show_back_face=True)   
    
    for y in range(6):
         FullListOfOutputs[y+(x*6)].translate([-1.5*y,0,+1.5*x])
        
  
    

In [27]:
GetVisualization(3,4)

In [28]:
GetGroupVisualization(0)
GetGroupVisualization(2)
GetGroupVisualization(3)

In [32]:
GetGroupVisualization(3)

In [33]:
import pandas as pd

In [34]:
    
def GetChamferDistance(ObjectIndex,TypeIndex,suppressOutputs=True):
    
    NumPoints = (np.asarray(FullListOfOutputs[(6*ObjectIndex)].points).shape[0])
    x = FullListOfOutputs[(6*ObjectIndex+1)].sample_points_uniformly(number_of_points=NumPoints)
    dists = np.asarray(FullListOfOutputs[(6*ObjectIndex)].compute_point_cloud_distance(x))
    if(not suppressOutputs):
        df = pd.DataFrame({"distances": dists}) 
        ax1 = df.boxplot(return_type="axes") 
        ax2 = df.plot(kind="hist", alpha=0.5, bins = 1000)
        ax3 = df.plot(kind="line") 
    plt.show()
    dists = abs((sum(dists)/NumPoints))
    return dists

In [35]:
from tabulate import tabulate
table = []
row1 = []
row1.insert(0,"Chamfer")
for x in range(1,6):
    row1.insert(x+1,"Method "+ str(x))
table.insert(0,row1)
for x in range(len(dataset_name)):
    row =[]
    row.insert(0,"PointCLoud "+ str(x))
    for y in range(1,6):
        k=0
        for z in range(50):
            k = k+GetChamferDistance(x,y)
        row.insert(y+1,k/50)
    table.insert(x+1,row)
print(tabulate(table,headers='firstrow'))

Chamfer         Method 1    Method 2    Method 3    Method 4    Method 5
------------  ----------  ----------  ----------  ----------  ----------
PointCLoud 0   0.0522318   0.0522309   0.0522304   0.0522326   0.0522295
PointCLoud 1   0.101186    0.101186    0.101189    0.101189    0.101187
PointCLoud 2   0.0626777   0.0626777   0.0626772   0.0626774   0.0626772
PointCLoud 3   0.057167    0.0571656   0.0571645   0.0571666   0.0571662


In [36]:
ReconstructionChoice = []
for x in range(len(dataset_name)):
#for x in range(1,2):
    row =[]
    for y in range(1,6):
        
        k=0
        for z in range(10):
            c=GetChamferDistance(x,y)
            k = k+c
        row.insert(y,k/10)
    print(row)
    ReconstructionChoice.insert(x,row.index(min(row))+1)
    print(row.index(min(row))+1)
    

[0.052235589540883545, 0.052230236994181015, 0.05223366724586974, 0.05223641070582943, 0.05223204210544589]
2
[0.1011872888015352, 0.10118887330145618, 0.10119283517268847, 0.10119372275552056, 0.10118384557526541]
5
[0.06267608534095079, 0.06267721890505755, 0.06267419812484562, 0.06267554929871541, 0.06267825355809245]
3
[0.05716697301813113, 0.05716561467657244, 0.05716566771772517, 0.057166524905951796, 0.05716742535383781]
2


In [37]:
for x in range(len(dataset_name)):
    GetVisualization(x,ReconstructionChoice[x])